In [1]:
#import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
from skimage import io
import random
from livelossplot import PlotLossesKeras

In [11]:
import tensorflow as tf
tf.test.gpu_device_name()
print (tf.__version__)
tf.config.experimental.list_physical_devices(device_type=None)

2.4.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# path to your dataset
Train_DATASET_PATH = './Updated/Processed/train'
TEST_DATASET = './Updated/Processed/test'
data_cls = ['COVID-19', 'NORMAL', 'Viral']

# globbing example
# help(glob)
covid_path = os.path.join(Train_DATASET_PATH, data_cls[0], '*')
print(covid_path)

./Updated/Processed/train\COVID-19\*


In [3]:
import numpy as np
import tensorflow as tf
# specify image size and channels
img_channels = 3
img_rows = 256
img_cols = 256

# number of classes
nb_classes = 3

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2,
                             rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1,
                             zoom_range=0.1,
                             horizontal_flip=True)

training_set = datagen.flow_from_directory(
        Train_DATASET_PATH,
        target_size=(256, 256),
        batch_size=32,shuffle=True,
        class_mode='categorical',
        subset='training')

validation_set = datagen.flow_from_directory(
    Train_DATASET_PATH,
    target_size=(256, 256),
    batch_size=32,shuffle=True,
    class_mode='categorical',
    subset='validation')

# validtion_datagen = ImageDataGenerator()
# validtion_set = validtion_datagen.flow_from_directory(
#         './Image_3/validation',
#         target_size=(224, 224),
#         batch_size=32,shuffle=True,
#         class_mode='categorical')


Found 2799 images belonging to 3 classes.
Found 699 images belonging to 3 classes.


In [15]:
# generic way to create custom callback
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))

In [16]:
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras import optimizers
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications import ResNet101, DenseNet201 ,InceptionV3
# from tensorflow.keras.layers import Input, Add, AveragePooling2D,  GlobalAveragePooling2D ,GlobalMaxPool2D
# from tensorflow.keras.callbacks import ModelCheckpoint

# input_shape = (224,224,3)
# densenet = tf.keras.applications.DenseNet121( include_top=False, weights=None, pooling='max',input_shape=input_shape )

# x = densenet.output
# predictions = Dense(3, activation="sigmoid", name="predictions")(x)
# model = Model(inputs=densenet.inputs, outputs=predictions)
# model.load_weights('./densenet121_weights_tf.h5',by_name=True, skip_mismatch=True)
# #print (model.summary())
# print (len(model.layers))

# base_model =  DenseNet201(weights='imagenet', include_top=False ,input_shape=input_shape)
# print (len(base_model.layers))

In [17]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet101, DenseNet201 ,InceptionV3, DenseNet121
from tensorflow.keras.layers import Input, Add, AveragePooling2D,  GlobalAveragePooling2D ,GlobalMaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint


def get_model(input_shape = (img_rows,img_cols,img_channels)):
    #Get base model: ResNet50 or VGG16

    base_model =  VGG19(weights='imagenet', include_top=False ,input_shape=input_shape)
    #base_model =  ResNet101(weights='imagenet', include_top=False ,input_shape=input_shape)    
    #base_model =  DenseNet201(weights='imagenet', include_top=False ,input_shape=input_shape)    
    #base_model =  InceptionV3(weights='imagenet', include_top=False ,input_shape=input_shape)    
    ## ChexNet
    #base_model = DenseNet121( include_top=False, weights=None, input_shape=input_shape )
    
#     split_at = 7
#     #print (len(base_model.layers))
#     for layer in base_model.layers[:split_at]: layer.trainable = False
#     for layer in base_model.layers[split_at:]: layer.trainable = True
    #for layer in base_model.layers: layer.trainable = False
        
    # Get the output from the base model 
    #base_model_output = base_model.output
    
    x = base_model.output
    x = AveragePooling2D()(x)
    x = Flatten()(x)
    predictions = Dense(3, activation= 'softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    #model.load_weights('./densenet121_weights_tf.h5',by_name=True, skip_mismatch=True)
    #model.load_weights('./brucechou1983_CheXNet_Keras_0.3.0_weights.h5',by_name=True, skip_mismatch=True)
    #Finetuning
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
#     for layer in base_model.layers:
#         layer.trainable = False
        
#     model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
#     # train the model on the new data for a few epochs
#     model.fit_generator(training_set,  epochs=2, validation_data=validation_set, validation_steps=21)
    
        # let's visualize layer names and layer indices to see how many layers
    # we should freeze:
#     for i, layer in enumerate(model.layers):
#        print(i, layer.name)
    
    # the first 248 layers and unfreeze the rest for inceptionv3: last 2 inception blocks
    # the first 480 layers and unfreeze the rest for Densenet201: last dense block
    # the first 312 layers and unfreeze the rest for resnet: last residual block
    # the first 10 layers and unfreeze the rest for VGG19: last 2 conv block
    split_at = 11
    for layer in model.layers[:split_at]:
       layer.trainable = False
    for layer in model.layers[split_at:]:
       layer.trainable = True
    
    
    print (model.summary())
    return model

model = get_model()
model.compile(loss='categorical_crossentropy',optimizer='SGD',
              metrics=['accuracy'])

# generators
#training_generator = DataGenerator('train')#, ablation=50)
#validation_generator = DataGenerator('val')#, ablation=50)
#print (training_generator)

history = LossHistory()
plot_data = {}
filepath = 'models_img/best_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# fit
model.fit_generator(training_set,  epochs=20, validation_data=validation_set, validation_steps=21, callbacks=[history, checkpoint])


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [18]:
print (len(history.losses))
print (history.losses)

20
[1.248996615409851, 0.3765992224216461, 0.24703018367290497, 0.21538293361663818, 0.22993548214435577, 0.16204464435577393, 0.1752534955739975, 0.1375114619731903, 0.13668470084667206, 0.1280728131532669, 0.12000250071287155, 0.12281256169080734, 0.10802589356899261, 0.09364466369152069, 0.09451119601726532, 0.09668582677841187, 0.09678496420383453, 0.11511107534170151, 0.07872350513935089, 0.08063217997550964]


In [5]:
#using the inbuilt imagegenerator
import h5py
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.math import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_set = test_datagen.flow_from_directory(
        TEST_DATASET,
        target_size=(256, 256),
        batch_size=32,shuffle=False,
        class_mode='categorical')

import matplotlib.pyplot as plt

test_set.reset() 

model = load_model('models_img/densenet201_Processed_3.hdf5')
#model = load_model('models_img/vgg_Image_4.hdf5')
Y_pred = model.predict_generator(test_set,388 // 32+1)
y_pred = np.argmax(Y_pred,axis=-1)
s=confusion_matrix(test_set.classes,y_pred)



Found 388 images belonging to 3 classes.


C:\tools\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [6]:
target_names = data_cls
print (test_set.classes)
print (y_pred)
c = classification_report(test_set.classes.tolist(),y_pred.tolist(),target_names=target_names)

print (s)
print (c)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0